In [1]:
from knossos_utils import knossosdataset, skeleton
import numpy as np
import neuroglancer
import os
import h5py

In [2]:
#Input of data Image path
knossos_cube = './kcube/knossos.conf'

#Input of data Image path
knossos_anno = './kcube/annotation-20190214T1833.000.k.zip'
    
#setting working magnification
mag = 1

#setting of loading box
box_size = [128,128,128]

#setting the location of the upper corner of the box in the original kcube
box_offset = [256,256,256]


##edit here to delete unwanted labels
delete_anno_low = 0 #will delete anything bellow #
delete_anno_high = 1000 #will delete anything above #


training_data_file = 'training_data'


##END OF INPUTS

In [3]:
##########DONT CHANGE
#intiializing virtual cube experiment
kd = knossosdataset.KnossosDataset()
kd.initialize_from_knossos_path(knossos_cube)


cube = kd.from_raw_cubes_to_matrix(
  size=box_size, 
  offset=box_offset)

print('load complete!')
print(cube.shape, cube.dtype)
print(cube.mean(), cube.std())
##########DONT CHANGE

Progress: finished
Speed: 147.319 MB or MPix /s, time 0.014235496520996094
load complete!
(128, 128, 128) uint8
126.22100448608398 41.172601119937475


/home/rlab/Documents/workspace/knossos_utils/knossos_utils/knossosdataset.py:1101: UserWarning: You are using implicit channel selection. This possibility will soon be removed. Please call set_channel() before reading or writing data using KnossosDataset.
  warnings.warn('You are using implicit channel selection. This possibility will soon be removed.'


In [4]:
##########DONT CHANGE
anno = kd.from_kzip_to_matrix(
  path=knossos_anno,
  size=box_size,
  offset=box_offset,
  mag=mag, 
  verbose=True)

print('load complete!')
print(anno.shape, anno.dtype)
##########DONT CHANGE

Progress: 100.00%knossos_cuber_project_mag1x2y2z2.seg.sz


applying mergelist now
Correct shape
load complete!
(128, 128, 128) uint64


Progress: finished
Speed: 36.108 MB or MPix /s, time 0.11077737808227539
Progress: 25.00%knossos_cuber_project_mag1_mag1x0y0z0.seg.sz
Cube does not exist, cube with 0 only assigned
Progress: 50.00%knossos_cuber_project_mag1_mag1x1y0z0.seg.sz
Cube does not exist, cube with 0 only assigned
Progress: 75.00%knossos_cuber_project_mag1_mag1x0y1z0.seg.sz
Cube does not exist, cube with 0 only assigned
Progress: 100.00%knossos_cuber_project_mag1_mag1x1y1z0.seg.sz
Cube does not exist, cube with 0 only assigned
applying mergelist now
Correct shape

In [5]:
print(cube.shape, cube.dtype)
print(anno.shape, anno.dtype)

anno[anno<delete_anno_low]  = 0
anno[anno>delete_anno_high] = 0


(128, 128, 128) uint8
(128, 128, 128) uint64


In [6]:
ids = np.unique(anno[...],return_counts=1)
print (ids)

(array([  0, 114, 121, 136, 185, 213, 255, 347, 437, 451, 478, 592, 636,
       640, 644, 660, 737], dtype=uint64), array([2073568,     628,    2472,    1615,     895,    1060,    2155,
          1294,    2806,    3381,     318,    1451,     141,    1009,
           641,     781,    2937]))


In [7]:
viewer = neuroglancer.Viewer()
with viewer.txn() as s:
  s.layers['image'] = neuroglancer.ImageLayer(
    source=neuroglancer.LocalVolume(data=cube, volume_type='image'))
  s.layers['labels'] = neuroglancer.SegmentationLayer(
    source=neuroglancer.LocalVolume(data=anno, volume_type='segmentation',mesh_options={'max_quadrics_error':100}),segments=ids[0])
print(viewer.get_viewer_url())

http://127.0.0.1:39293/v/47d3591b47fbbd6066db7491fecde08bb442177f/


In [8]:
del viewer

In [9]:

#this will create the training data for FFN from knossos files
#change here for the training data filename



#DONT TOUCH
labels = anno.astype('int64')

print ("Working Dir")
print (os.getcwd())


print ('Cube Properties!')
print (cube.dtype)
print (cube.shape)

print ('Mean : '+str(cube.mean()))
print ('Std : '+str(cube.std()))

print ('Labels Properties!')
print (labels.dtype)
print (labels.shape)

print ('Ids Properties!')
ids = np.unique(labels,return_counts=1)
print (ids)


h5file = h5py.File(training_data_file+'.h5', 'w')
h5file.create_dataset('image',data=cube)
h5file.create_dataset('labels',data=labels)
h5file.close()

print ("Finished!! Goodbye!!")
#DONT TOUCH

Working Dir
/media/Trantor2/Public/ffn_test_goodpeople
Cube Properties!
uint8
(128, 128, 128)
Mean : 126.22100448608398
Std : 41.172601119937475
Labels Properties!
int64
(128, 128, 128)
Ids Properties!
(array([  0, 114, 121, 136, 185, 213, 255, 347, 437, 451, 478, 592, 636,
       640, 644, 660, 737]), array([2073568,     628,    2472,    1615,     895,    1060,    2155,
          1294,    2806,    3381,     318,    1451,     141,    1009,
           641,     781,    2937]))
Finished!! Goodbye!!
